In [ ]:
import numpy as np
import pandas as pd
import sqlite3
import matplotlib as plt
%matplotlib inline
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.holtwinters import SimpleExpSmoothing

In [ ]:
cnx = sqlite3.connect('../out/binance_historic_trades.db')

In [ ]:
df = pd.read_sql_query("SELECT * FROM historic_trades", cnx)

In [ ]:
df['timestamp'] = df['timestamp'].drop_duplicates(keep='first')

In [ ]:
df.dropna(subset=['timestamp'], how='all', inplace=True)

In [ ]:
df['timestamp'] = pd.to_datetime(df['timestamp'], unit = 'ms')

In [ ]:
df = df.set_index('timestamp')

In [ ]:
df = df.drop(['tradeId', 'symbol', 'quantity'], axis = 1)

In [ ]:
df.resample(rule = 'S')

In [ ]:
df = df.asfreq('S')

In [ ]:
df['price'] = df['price'].fillna(method='ffill')

In [ ]:
df['TESadd12'] = ExponentialSmoothing(df['price'],trend='add',seasonal='add',seasonal_periods = 2).fit().fittedvalues
df.head()

In [ ]:
df[['price','TESadd12']].plot(figsize=(12,6)).autoscale(axis='x',tight=True);

In [ ]:
train_data = df.iloc[:152707] # 75%
test_data = df.iloc[152707:]

In [ ]:
fitted_model = ExponentialSmoothing(train_data['price'],trend='mul',seasonal='mul', seasonal_periods=2).fit()

In [ ]:
test_predictions = fitted_model.forecast(3600).rename('HW Forecast')

In [ ]:
train_data['price'].plot(legend=True,label='TRAIN')
test_data['price'].plot(legend=True,label='TEST',figsize=(12,8))
test_predictions.plot(legend=True,label='PREDICTION');